In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam

In [2]:
import os
import random

import cv2

path_to_folder1 = r'D:\Norm\Norm'  
path_to_folder2  = r'D:\pat\pat'


image_size = (512, 512) 
num_channels = 3  


category1_files = [os.path.join(path_to_folder1, f) for f in os.listdir(path_to_folder1)[::60] if f.endswith('.jpg')]
category1_labels = [0] * len(category1_files)  

category2_files = [os.path.join(path_to_folder2, f) for f in os.listdir(path_to_folder2)[::60] if f.endswith('.jpg')]
category2_labels = [1] * len(category2_files) 

all_files = category1_files + category2_files
all_labels = category1_labels + category2_labels


combined_data = list(zip(all_files, all_labels))
random.shuffle(combined_data)
all_files, all_labels = zip(*combined_data)

num_samples = len(all_files)
dataset_images = np.zeros((num_samples, image_size[0], image_size[1], num_channels), dtype=np.float32)
dataset_labels = np.array(all_labels)

for i, image_path in enumerate(all_files):
    
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image.astype(np.float32) / 255.0

    
    dataset_images[i] = image


print("Размер датасета:", dataset_images.shape)
print("Метки:", dataset_labels)

Размер датасета: (211, 512, 512, 3)
Метки: [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1
 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0]


In [3]:
dataset_images

array([[[[0.9882353 , 0.9882353 , 0.9882353 ],
         [0.9882353 , 0.9882353 , 0.9882353 ],
         [0.9882353 , 0.9882353 , 0.9882353 ],
         ...,
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843]],

        [[0.9882353 , 0.9882353 , 0.9882353 ],
         [0.9882353 , 0.9882353 , 0.9882353 ],
         [0.9882353 , 0.9882353 , 0.9882353 ],
         ...,
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843]],

        [[0.9882353 , 0.9882353 , 0.9882353 ],
         [0.9882353 , 0.9882353 , 0.9882353 ],
         [0.99215686, 0.99215686, 0.99215686],
         ...,
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843]],

        ...,

        [[0.8117647 , 0.8117647 , 0.8117647 ],
         [0.7882353 , 0.7882353 , 0.7882353 ]

In [6]:
from sklearn.model_selection import train_test_split

from PIL import Image
X_train, X_test, y_train, y_test = train_test_split(dataset_images, dataset_labels, test_size=0.2, random_state=42)

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(dataset_images, dataset_labels))

Epoch 1/5
6/6 [==============================] - 482s 78s/step - loss: 1.9431 - accuracy: 0.6190 - val_loss: 0.6170 - val_accuracy: 0.7630
Epoch 2/5
6/6 [==============================] - 420s 71s/step - loss: 0.5954 - accuracy: 0.7857 - val_loss: 0.5497 - val_accuracy: 0.7630
Epoch 3/5
6/6 [==============================] - 441s 75s/step - loss: 0.5074 - accuracy: 0.7857 - val_loss: 0.5226 - val_accuracy: 0.7630
Epoch 4/5
6/6 [==============================] - 447s 77s/step - loss: 0.4937 - accuracy: 0.7857 - val_loss: 0.4687 - val_accuracy: 0.7630
Epoch 5/5
6/6 [==============================] - 384s 62s/step - loss: 0.4464 - accuracy: 0.8155 - val_loss: 0.3898 - val_accuracy: 0.8152


In [27]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
6/6 [==============================] - 352s 58s/step - loss: 0.4697 - accuracy: 0.7321 - val_loss: 0.7477 - val_accuracy: 0.6744
Epoch 2/5
6/6 [==============================] - 330s 53s/step - loss: 0.3730 - accuracy: 0.8036 - val_loss: 0.4916 - val_accuracy: 0.6744
Epoch 3/5
6/6 [==============================] - 321s 53s/step - loss: 0.3248 - accuracy: 0.8571 - val_loss: 0.7543 - val_accuracy: 0.7209
Epoch 4/5
6/6 [==============================] - 368s 59s/step - loss: 0.2810 - accuracy: 0.8690 - val_loss: 0.4567 - val_accuracy: 0.8372
Epoch 5/5
6/6 [==============================] - 369s 59s/step - loss: 0.2478 - accuracy: 0.9286 - val_loss: 0.6356 - val_accuracy: 0.8140


In [29]:
model.save('model_vector_valid1.h5')

In [3]:
path_to_folder1 = r"D:\Validation\Validation"


image_size = (512, 512)  
num_channels = 3  


category1_files = [os.path.join(path_to_folder1, f) for f in os.listdir(path_to_folder1) if f.endswith('.jpg')]
category1_labels = [0] * len(category1_files) 


all_files = category1_files 
all_labels = category1_labels 


num_samples = len(all_files)
Validation_images = np.zeros((num_samples, image_size[0], image_size[1], num_channels), dtype=np.float32)
Validation_labels = np.array(all_labels)

for i, image_path in enumerate(all_files):
    
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image.astype(np.float32) / 255.0

    
    Validation_images[i] = image


print("Размер датасета:", Validation_images.shape)


Размер датасета: (1000, 512, 512, 3)


In [2]:
import tensorflow as tf


model = tf.keras.models.load_model('model_vector_valid.h5')


In [9]:

validation_predictions = model.predict(Validation_images)

32/32 [==============================] - 145s 4s/step


In [19]:
results = pd.DataFrame({'image_id': all_files, 'activation': np.squeeze(validation_predictions)})

In [20]:
results

,image_id,activation
0,D:\Validation\Validation\2000.jpg,3.071074e-01
1,D:\Validation\Validation\2001.jpg,9.474734e-05
2,D:\Validation\Validation\2002.jpg,9.992850e-01
3,D:\Validation\Validation\2003.jpg,9.115705e-08
4,D:\Validation\Validation\2004.jpg,4.054257e-02
...,...,...
995,D:\Validation\Validation\3096.jpg,5.949343e-03
996,D:\Validation\Validation\3097.jpg,3.919413e-01
997,D:\Validation\Validation\3098.jpg,9.712991e-01
998,D:\Validation\Validation\3099.jpg,1.194288e-01


In [21]:
results.to_csv('results_validation_n.csv', index=False)